# <center>Block 9: estimation of matching models</center>
### <center>Alfred Galichon (NYU & Sciences Po)</center>
## <center>'math+econ+code' masterclass on optimal transport and economic applications</center>
<center>© 2018-2021 by Alfred Galichon. Past and present support from NSF grant DMS-1716489, ERC grant CoG-866274, and contributions by Jules Baudet, Pauline Corblet, Gregory Dannay, and James Nesbit are acknowledged.</center>

#### <center>With python code examples</center>

## Learning objectives

* Matching with unobserved heterogeneities

* Estimation of matching models

## References

**[B]** Becker (1973). 'A Theory of Marriage: Part 1.' *Journal of Political Economy*.

**[COQ]** Chiappori, Oreffice and Quintana-Domeque (2012). 'Fatter Attraction: Anthropometric and Socioeconomic Matching on the Marriage Market'. *Journal of Political Economy*.

**[CS]** Choo and Siow (2006). 'Who Marries Whom and Why'. *Journal of Political Economy*.

**[CSW]** Chiappori, Salanié, and Weiss (2017). 'Partner Choice and the Marital College Premium'. * American Economic Review*.

**[DG]** Dupuy and Galichon (2014). 'Personality traits and the marriage market'. *Journal of Political Economy*.

**[GS]** Galichon and Salanié (2020). 'Cupid's Invisible Hand: Social Surplus and Identification in Matching Models'. Preprint (first version 2011).




# Motivation: models of matching since Gary Becker

* In the footsteps of Becker, empirical studies on the marriage market had long been focused on one-dimensional models, which assumes that a single index is enough to capture the interactions on the marriage market, and positive assortative matching (PAM), which predicts that the matching equilibrium will tend to match the agents with higher indices with each other.

* However, it is desirable to move beyond PAM:
    * PAM is always loosely true, never precisely
    * there are often many observed characteristics, and it is not always the case that the sorting can be captured by a single-dimensional model
    * PAM is a theoretical prediction stemming from assumptions of supermodularity of the surplus function which do not necessarly hold
    * optimal transport provide tools to study multidimensional models


* However, any model of matching based on (unregularized) optimal transport will not be exploitable because it will generate far too strong predictions, namely that some matchings will never hold. This is rather counterfactual: in the data, one observes virtually any combination of type.

* Hence, need to regularize the matching model, and we shall do so by introducing unobserved heterogeneity. The model so obtained will be exploitable for estimation and identification purposes. The first such model (with transfers) is the model by [CS]. We shall see a generalization of this model by [GS] (2015).


## Loading our libraries

We start with loading the libraries we will need. They are rather standard.

In [1]:
import gurobipy as grb
import pandas as pd
import numpy as np
import os
from scipy import optimize

## A look at our data

* Our data are Choo and Siow's original data. Choo and Siow wanted to study the impact of the legalization of abortion by the Roe vs. Wade decision by the supreme court on the 'value of marriage'. Roe vs. Wade decreased the role of marriage in covering out-of-the-wedlocks pregnancies ('shotgun weddings').

* The decision did, however, not make a change uniformly in the United
States as a number of states had already legalized abortion (reform states).
Choo and Siow thus offer a diffs-in-diffs approach in order to compute the
change in the value of marriage.

* Choo and Siow's data are thus made of the marriages between men and women in reformed states (R) vs nonreformed states (NR), in 1972 and in 1982. One should expect to see a higher drop in marriage value in NR states.

In [2]:
thepath = os.getcwd()
n_singles = pd.read_csv(os.path.join(thepath,'data_mec_optim/marriage-ChooSiow/n_singles.txt'), sep='\t', header = None)
marr = pd.read_csv(os.path.join(thepath,'data_mec_optim/marriage-ChooSiow/marr.txt'), sep='\t', header = None)
navail = pd.read_csv(os.path.join(thepath,'data_mec_optim/marriage-ChooSiow/n_avail.txt'), sep='\t', header = None)

The data used by Choo and Siow is census data on marriages between age categories, from age 16 (row/column 0) to age 75 (row/age 59). It is thus 60x60 tables:

In [3]:
marr.head()

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
0,22704,10954,3932,1550,672,414,190,114,78,64,...,0,0,0,0,0,0,0,0,0,0
1,40266,38980,16368,5714,2116,1101,691,427,260,127,...,0,0,0,0,0,0,0,0,0,0
2,39219,49753,40315,18001,5401,2291,1429,856,611,338,...,0,0,0,0,0,0,0,0,0,0
3,29500,44788,47212,39429,16730,6270,3211,1910,762,438,...,0,0,0,0,0,0,0,0,0,0
4,18952,32860,40103,43488,36265,15318,6076,2872,1552,1246,...,0,0,0,0,0,0,0,0,0,0


The data also includes the number of single individuals per age category:

In [4]:
n_singles.transpose().head()

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
0,1010132,907226,772448,597919,454792,341370,315797,253618,171614,152228,...,68383,69177,61358,70752,69112,60909,60749,65302,62922,61117
1,790793,671332,602900,492020,404882,313755,255414,195418,141283,128787,...,197456,201183,191519,218031,219157,199926,200052,202146,210315,202775


# Building the model

The analysis here follows [GS], who build on the logit model by [CS]. 

* Consider a heterosexual marriage matching market. The set of types (observable characteristics) is $\mathcal{X}$ for men, and $\mathcal{Y}$ for women. There are $n_{x}$ men of type $x$, and $m_{y}$ women of type $y$.

* Assume that if a man $i\in\mathcal{I}$ of type $x_{i}$ and a woman $j\in\mathcal{J}$ of type $y_{j}$ match, they get respective utilities \begin{align*} &  \alpha_{x_{i}y_{j}}+w_{ij}+\varepsilon_{iy_{j}}\\ &  \gamma_{x_{i}y_{j}}-w_{ij}+\eta_{x_{i}j} \end{align*} where $w_{ij}$ is the transfer from $i$ to $j$. If they remain single $i$ and $j$ get respectively $\varepsilon_{i0}$ and $\eta_{0j}$.

* The random utility vectors $\left(  \varepsilon_{y}\right)  $ and $\left(  \eta_{x}\right)  $ are drawn from probability distributions $\mathbf{P}_{x}$ and $\mathbf{Q}_{y}$, respectively. In the sequel we shall work with a finite number of agents of each type, and then we'll investigate the limit of these results.

* The fact that the preferences for the other side of the market terms $\varepsilon_{iy_{j}}$ and $\eta_{x_{i}}$ do not vary within observed types is a very important implicit assumption called **separability**.  


## Optimal matching


The matching surplus between $i$ and $j$ is therefore $$\tilde{\Phi}_{ij}=\Phi_{x_{i}y_{j}}+\varepsilon_{iy_{j}}+\eta_{x_{i}j}$$ where $\Phi_{xy}=\alpha_{xy}+\gamma_{xy}$. The value of optimal matching is thus, under its dual form, \begin{align*}\min_{u_{i},v_{j}}  &  \sum_{i\in\mathcal{I}}u_{i}+\sum_{j\in\mathcal{J}}  v_{j}\\ s.t.~  &  u_{i}+v_{j}\geq\Phi_{x_{i}y_{j}}+\varepsilon_{iy_{j}}+\eta_{x_{i}j}\\ &  u_{i}\geq\varepsilon_{i0}\\ &  v_{j}\geq\eta_{j0} \end{align*}

Written like this, the lp has $\left\vert \mathcal{I}\right\vert +\left\vert \mathcal{J}\right\vert $ variables and $\left\vert \mathcal{I} \right\vert \times\left\vert \mathcal{J}\right\vert +\left\vert \mathcal{I} \right\vert +\left\vert \mathcal{J}\right\vert $ constraints. Assuming that there are $K$ individuals per type for each type, this is $K\left(  \left\vert \mathcal{X}\right\vert +\left\vert \mathcal{Y}\right\vert \right)  $ variables and $K^{2}\left(  \left\vert \mathcal{X}\right\vert \times\left\vert \mathcal{Y}\right\vert \right)  +K\left(  \left\vert \mathcal{X}\right\vert +\left\vert \mathcal{Y}\right\vert \right)  $ constraints.

The number of constraints is **quadratic** with respect to $K$. Fortunately, a little thinking about the implications of separability will help us reduce this complexity.


## A property of equilibrium

We have:

---

**Lemma**. Consider the set $\mathcal{I}_{xy}$ of men of type $x$ matched to women of type $y$ at equilibrium. If $\mathcal{I}_{xy}$ is nonempty, then $u_{i}-\varepsilon_{iy}$ is a constant across $\mathcal{I}_{xy}$.

---

**Proof**. For $i\in\mathcal{I}$ such that $x_{i}=x$,
\begin{align*}
u_{i}  &  =\max_{j\in\mathcal{J}}\left\{  \tilde{\Phi}_{ij}-v_{j}%
,\varepsilon_{i0}\right\} \\
&  =\max_{y\in\mathcal{Y}}\left\{  U_{xy}+\varepsilon_{iy},\varepsilon
_{i0}\right\}
\end{align*}
where $U_{xy}=\max_{j:y_{j}=y}\left\{  \Phi_{xy}+\eta_{x_{i}j}-v_{j}\right\}
$, thus $u_{i}\geq U_{xy}+\varepsilon_{iy}$ with equality on $\mathcal{I}%
_{xy}$. With similar notations, $v_{j}\geq V_{xy}+\eta_{xj}$ with equality on
$\mathcal{J}_{xy}$. As a result, if $\mathcal{I}_{xy}$ is nonempty, then
$U_{xy}+V_{xy}=\Phi_{xy}$ and $\forall i\in\mathcal{I}_{xy},$ $u_{i}%
=U_{xy}+\varepsilon_{iy}$.



## A simplification

In the sequel, we shall see that *adding* an auxiliary variable to
the previous lp will lead to *decreasing* the computational complexity of
the problem.

Observe that the first set of constraints is reexpressed by saying that,
for every $x\in\mathcal{X}$, $y\in\mathcal{Y}$,
$$
\min_{i:x_{i}=x}\left\{  u_{i}-\varepsilon_{iy}\right\}  +\min_{j:y_{j}
=y}\left\{  v_{j}-\eta_{xj}\right\}  \geq\Phi_{xy}.
$$


Hence, letting $U_{xy}=\min_{i:x_{i}=x}\left\{  u_{i}-\varepsilon
_{iy}\right\}  $ and $V_{xy}=\min_{j:y_{j}=y}\left\{  v_{j}-\eta_{xj}\right\}
$, a solution of the previous lp should satisfy
$$
u_{i}=\max_{y\in\mathcal{Y}}\left\{  U_{xy}+\varepsilon_{iy},\varepsilon
_{i0}\right\}  \text{ and }v_{j}=\max_{x\in\mathcal{X}}\left\{  V_{xy}
+\varepsilon_{xj},\varepsilon_{0j}\right\}  .
$$



The problem rewrites as
\begin{align}
\min_{u_{i},v_{j},U_{xy},V_{xy}}  &  \sum_{i\in\mathcal{I}}u_{i}+\sum
_{j\in\mathcal{J}}v_{j}\label{simplifiedDual}\\
s.t.~  &  U_{xy}+V_{xy}\geq\Phi_{xy}~\left[  \mu_{xy}\geq0\right] \nonumber\\
&  u_{i}\geq U_{x_{i}y}+\varepsilon_{iy_{j}}~\left[  \mu_{iy}\right]
\nonumber\\
&  v_{j}\geq V_{xy_{j}}+\eta_{x_{i}j}~\left[  \mu_{xi}\right] \nonumber\\
&  u_{i}\geq\varepsilon_{i0}~\left[  \mu_{i0}\right] \nonumber\\
&  v_{j}\geq\eta_{j0}~\left[  \mu_{0x}\right] \nonumber
\end{align}

This problem has $K\left(  \left\vert \mathcal{X}\right\vert +\left\vert
\mathcal{Y}\right\vert \right)  +\left\vert \mathcal{X}\right\vert
\times\left\vert \mathcal{Y}\right\vert $ variables and $\left(  \left\vert
\mathcal{X}\right\vert \times\left\vert \mathcal{Y}\right\vert \right)
+K\left(  2\left\vert \mathcal{X}\right\vert \times\left\vert \mathcal{Y}
\right\vert +\left\vert \mathcal{X}\right\vert +\left\vert \mathcal{Y}
\right\vert \right)  $ constraints.

The number of constraint is now **linear** with respect to $K$.

## Consequences

**1. Lagrange multipliers:**
* The Lagrange multiplier $\mu_{xy}$ is interpreted as the number of matchings between types $x$ and $y$.

* The Lagrange multiplier $\mu_{iy}$ ($y\in\mathcal{Y}_{0}$) is interpreted as a 0-1 indicator that man $i$ chooses a type $y$

* The Lagrange multiplier $\mu_{xj}$ ($x\in\mathcal{X}_{0}$) is interpreted as a 0-1 indicator that woman $j$ chooses a\ type $x$

**2. Utilities:**
* Man $i$ solves a discrete choice problem $u_{i}=\max_{y\in\mathcal{Y}
}\left\{  U_{xy}+\varepsilon_{iy},\varepsilon_{i0}\right\}  $

* Woman $j$ solve a discrete choice problem $v_{j}=\max_{x\in\mathcal{X}
}\left\{  V_{xy}+\eta_{xj},\eta_{0j}\right\}  .$
\end{itemize}

* $U_{xy}$ and $V_{xy}$ are related by $U_{xy}+V_{xy}\geq\Phi_{xy}$ with
equality if $\mu_{xy}>0$.


## Large market limit

Now look at the limit of previous markets when the number of market participants gets large, holding fixed the frequency of each types.

In the large population limit $n_{x}$ and $m_{y}$ are now interpreted as the mass distribution of respective types $x$ and $y$.

We shall from now on assume that $\mathbf{P}_{x}$ and $\mathbf{Q}_{y}$, the distributions of random utility vectors $\left( \varepsilon_{y}\right)  $ and $\left(  \eta_{x}\right)  $, have a density with full support. This will ensure that the Emax operators associated with the choice problems of the men and the women respectively \begin{align*} & G_x(U_{x.}) = \mathbb{E}_\mathbf{P} \left[\max_{y\in\mathcal{Y}
}\left\{  U_{xy}+\varepsilon_{iy},\varepsilon_{i0}\right\} \right]\text{, and }\\ & H_y(V_{.y}) = \mathbb{E}_\mathbf{Q} \left[\max_{x\in\mathcal{X}
}\left\{  V_{xy}+\eta_{xj},\eta_{0j}\right\} \right],\end{align*}as well as the corresponding entropies of choice $G_x^{\ast}$ and $H_y^{\ast}$ are continuously differentiable.

Under these assumptions, the problem becomes
\begin{align*}
\min_{U,V} ~&  G\left(  U\right)  +H\left(  V\right) \\
s.t.~  &  U_{xy}+V_{xy}\geq\Phi_{xy}~\left[  \mu_{xy}\right]
\end{align*}
where
\begin{align*}
G\left(  U\right)   &  =\sum_{x\in\mathcal{X}}n_{x}\mathbb{E}_{\mathbf{P}%
}\left[  \max_{y\in\mathcal{Y}}\left\{  U_{xy}+\varepsilon_{iy},\varepsilon
_{i0}\right\}  \right] \\
H\left(  V\right)   &  =\sum_{y\in\mathcal{Y}}m_{y}\mathbb{E}_{\mathbf{Q}%
}\left[  \max_{x\in\mathcal{X}}\left\{  V_{xy}+\eta_{xj},\eta_{0j}\right\}
\right]
\end{align*}


By first order conditions,
$$
\frac{\partial G\left(  U\right)  }{\partial U_{xy}}=\mu_{xy}=\frac{\partial
H\left(  V\right)  }{\partial V_{xy}}.
$$
and $\mu_{xy}>0$ for every $x\in\mathcal{X}$ and $y\in\mathcal{Y}$.



## Social planner's problem

The primal problem corresponding the problem above is
$$
\max_{\mu_{xy}\geq0}\sum_{\substack{x\in\mathcal{X}\\y\in\mathcal{Y}}}\mu
_{xy}\Phi_{xy}-\mathcal{E}\left(  \mu\right)
$$
where
$$
\mathcal{E}\left(  \mu\right)  =G^{\ast}\left(  \mu\right)  +H^{\ast}\left(
\mu\right)
$$


Recall $G^{\ast}\left(  \mu\right)  =\max\left\{  \sum_{xy}\mu
_{xy}U_{xy}-G\left(  U\right)  \right\}  $ is the Legendre transform of $G$, and similarly for $H^{\ast}$.

## Identification of the matching surplus



---

**Theorem.** By first order conditions, we get the identifcation formula of $\Phi$%
$$
\Phi_{xy}=\frac{\partial G^{\ast}\left(  \mu\right)  }{\partial\mu_{xy}}
+\frac{\partial H^{\ast}\left(  \mu\right)  }{\partial\mu_{xy}}.
$$

---

This means that the surplus function is identified *nonparametrically* given the matching patterns $\mu$ and assuming a fixed distribution of unobserved heterogeneity.

Hence only the joint surplus $\Phi
_{xy}=\alpha_{xy}+\gamma_{xy}$ is identified. However, if the transfers
$\hat{w}_{xy}$ are observed too (e.g. wages in labour market), then
$U_{xy}=\alpha_{xy}+w_{xy}$ and $V_{xy}=\gamma_{xy}-w_{xy}$, so that $\alpha$
and $\gamma$ are separately identified by
$$
\left\{
\begin{array}
[c]{c}%
\hat{\alpha}_{xy}=\frac{\partial G^{\ast}\left(  \mu\right)  }{\partial\mu_{xy}}-\hat{w}_{xy}\\
\hat{\gamma}_{xy}=\frac{\partial H^{\ast}\left(  \mu\right)  }{\partial\mu_{xy}}+\hat{w}_{xy}%
\end{array}
\right.
$$


## Choo and Siow's logit model

In Choo and Siow's model [CS], the heterogeneities in tastes are Gubmel,
we have
$$
\mathcal{E}\left(  \mu\right)  =2\sum_{\substack{x\in\mathcal{X}%
\\y\in\mathcal{Y}}}\mu_{xy}\log\mu_{xy}+\sum_{x\in\mathcal{X}}\mu_{x0}\log
\mu_{x0}+\sum_{y\in\mathcal{Y}}\mu_{0y}\log\mu_{0y}.
$$
Note that $\mathcal{E}\left(  \mu\right)  < + \infty$ if and only if $\mu
\in\mathcal{M}\left(  n,m\right)  $.


By first order conditions above, Choo-Siow's TU-logit model implies the
following matching function:
$$
\mu_{xy}=M_{xy}\left(  \mu_{x0},\mu_{0y}\right)  :=\sqrt{\mu_{x0}}\sqrt
{\mu_{0y}}\exp\left(  \frac{\Phi_{xy}}{2}\right) 
$$

This is a gravity equation of sorts. The full link with gravity equations is explored in the next lecture. 

As a result, $\partial\mathcal{E}\left(  \mu\right)  /\partial\mu
_{xy}=2\log\mu_{xy}-\log\mu_{x0}-\log\mu_{0y}$, which implies that $\Phi_{xy}$
is estimated by *Choo and Siow's identification formula*
$$
\hat{\Phi}_{xy}=\log\frac{\hat{\mu}_{xy}^{2}}{\hat{\mu}_{x0}\hat{\mu}_{0y}}.
$$

## Solving equilibrium in the Choo-Siow model

Write down the equilibrium equations in the TU-logit model:
$$
\left\{
\begin{array}
[c]{c}
\sum_{y\in\mathcal{Y}}\sqrt{\mu_{x0}}\sqrt{\mu_{0y}}\exp\left(  \frac
{\Phi_{xy}}{2}\right)  +\mu_{x0}=n_{x}\\
\sum_{x\in\mathcal{X}}\sqrt{\mu_{x0}}\sqrt{\mu_{0y}}\exp\left(  \frac
{\Phi_{xy}}{2}\right)  +\mu_{0y}=m_{y}%
\end{array}
\right.
$$


Setting $a_{x}=\sqrt{\mu_{x0}}$, $b_{y}=\sqrt{\mu_{0y}}$, and
$K_{xy}=\exp\left(  \Phi_{xy}/2\right)  $, this rewrites as
$$
\left\{
\begin{array}
[c]{c}
\sum_{y\in\mathcal{Y}}K_{xy}a_{x}b_{y}+a_{x}^{2}=n_{x}\\
\sum_{x\in\mathcal{X}}K_{xy}a_{x}b_{y}+b_{y}^{2}=m_{y}%
\end{array}
\right.$$

which is a variant of the equations previously seen to accomodate unmatched agents.

We can easily adapt the IPFP to this setting. The IPFP will consists in iteratively solving quadratic equations:
$$
\left\{
\begin{array}
[c]{l}
a_{x}^{2t+1}=\sqrt{n_{x}+\left(  \sum_{y\in\mathcal{Y}}b_{y}^{2t}%
K_{xy}/2\right)  ^{2}}-\sum_{y\in\mathcal{Y}}b_{y}^{2t}K_{xy}/2\\
b_{y}^{2t+2}=\sqrt{m_{y}+\left(  \sum_{x\in\mathcal{X}}a_{x}^{2t+1}%
K_{xy}/2\right)  ^{2}}-\sum_{x\in\mathcal{X}}a_{x}^{2t+1}K_{xy}/2
\end{array}
\right.
$$

## Dual problem

The dual problem is given by
$$
\min_{u,v}\left\{
\begin{array}
[c]{c}%
\sum_{x}n_{x}u_{x}+\sum_{y}m_{y}v_{y}\\
+2\sum_{xy}\sqrt{n_{x}m_{y}}\exp\left(  \frac{\Phi_{xy}-u_{x}-v_{y}}{2}\right)
\\
+\sum_{x}n_{x}\exp\left(  -u_{x}\right)  +\sum_{y}m_{y}\exp\left(
-v_{y}\right)
\end{array}
\right\}
$$


**Remarks.**

* This problem is an unconstrained convex optimization problem, so this formulation will be quite useful.

* If $\left(  u,v\right)  $ is solution, $u_{x}=-\log\mu_{0|x}=-\log\left(  \mu_{x0}/n_{x}\right)  $ and $v_{y}=-\log\left(  \mu
_{0|y}\right)  $.

* Note that the IPFP algorithm just seen interprets as (blockwise) *coordinate descent* method in the dual problem.


### Another application: estimation of affinity matrix

Dupuy and G (2014) focus on cross-dimensional interactions

\begin{align*}
\phi_{xy}^{A}=\sum_{p,q}A_{pq}\xi_{x}^{p}\xi_{y}^{q}
\end{align*}

and estimate "affinity matrix" $A$ on a dataset of married individuals where the "big 5" personality traits are measured.

$A$ is estimated by

\begin{align*}
\min_{s_{i},m_{n}}\min_{A}\left\{
\begin{array}
[c]{c}%
\sum_{x}p_{x}u_{x}+\sum_{y}q_{y}v_{y}\\
+\sum_{xy}\exp\left(  \sum_{p,q}A_{pq}\xi_{x}^{p}\xi_{y}^{q}-u_{x}%
-v_{y}\right) \\
-\sum_{x,y,p,q}\hat{\pi}_{xy}A_{pq}\xi_{x}^{p}\xi_{y}^{q}%
\end{array}
\right\}  .
\end{align*}

Dupuy, Galichon and Sun (2016) consider the case when the space of characteristics is high-dimensional. More on this soon.

### Estimation of affinity matrix: results

|  Husbands   \ Wives             | Education | Height | BMI   | Health | Consc. | Extra. | Agree | Emotio | Auto. | Risk  |
|-------------------|-----------|--------|-------|--------|--------|--------|-------|--------|-------|-------|
| Education         | 0.46      | 0      | -0.06 | 0.01   | -0.02  | 0.03   | -0.01 | -0.03  | 0.04  | 0.01  |
| Height            | 0.04      | 0.21   | 0.04  | 0.03   | -0.06  | 0.03   | 0.02  | 0      | -0.01 | 0.02  |
| BMI               | -0.03     | 0.03   | 0.21  | 0.01   | 0.03   | 0      | -0.05 | 0.02   | 0.01  | -0.02 |
| Health            | -0.02     | 0.02   | -0.04 | 0.17   | -0.04  | 0.02   | -0.01 | 0.01   | 0     | 0.03  |
| Conscienciousness | -0.07     | -0.01  | 0.07  | 0      | 0.16   | 0.05   | 0.04  | 0.06   | 0.01  | 0.01  |
| Extraversion      | 0         | -0.01  | 0     | 0.01   | -0.06  | 0.08   | -0.04 | -0.01  | 0.02  | -0.06 |
| Agreeableness     | 0.01      | 0.01   | -0.06 | 0.02   | 0.1    | -0.11  | 0     | 0.07   | -0.07 | -0.05 |
| Emotional         | 0.03      | -0.01  | 0.04  | 0.06   | 0.19   | 0.04   | 0.01  | -0.04  | 0.08  | 0.05  |
| Autonomy          | 0.03      | 0.02   | 0.01  | 0.02   | -0.09  | 0.09   | -0.04 | 0.02   | -0.1  | 0.03  |
| Risk              | 0.03      | -0.01  | -0.03 | -0.01  | 0      | -0.02  | -0.03 | -0.03  | 0.08  | 0.14  |

Affinity matrix. Source: Dupuy and G (2014). Note: Bold coefficients are significant at the 5 percent level.

# Choo-Siow application

In [5]:
nbCateg = 25

muhat_x0 = n_singles[0].iloc[0:nbCateg]
muhat_0y = n_singles[1].iloc[0:nbCateg]
muhat_xy = marr.iloc[0:nbCateg:,0:nbCateg]

In [7]:
Nh = muhat_xy.values.sum()+muhat_x0.sum()+muhat_0y.sum()

In [8]:
2*muhat_xy.values.sum()+muhat_x0.sum()+muhat_0y.sum()

14885023

In [9]:
muhat_xy = muhat_xy / Nh 
muhat_x0 = muhat_x0 / Nh 
muhat_0y = muhat_0y / Nh

In [10]:
n_x = muhat_xy.sum(axis = 1)+muhat_x0
m_y = muhat_xy.sum(axis = 0)+muhat_0y

In [11]:
nbX = nbCateg
nbY = nbCateg

xs = np.repeat(range(1,nbX+1),nbY).reshape(nbX,nbY)/25
ys = np.repeat(range(1,nbY+1),nbX).reshape(nbX,nbY).T/25

phi1_xy = -((xs-ys)**2).flatten()
phimat = np.column_stack((phi1_xy,np.multiply(phi1_xy,(((xs+ys)/2)**2).flatten()),np.multiply(phi1_xy,(((xs+ys-2)/2)**2).flatten()),np.multiply(phi1_xy,((xs+ys-1)**2).flatten())))

In [12]:
nbK = phimat.shape[1]
phimat_mean = phimat.mean(axis = 0)
phimat_stdev = phimat.std(axis = 0, ddof = 1)
phimat = ((phimat - phimat_mean).T/phimat_stdev[:,None]).T

In [13]:
def ObjFunc(uvlambda):
    u_x = uvlambda[0:nbX]
    v_y = uvlambda[nbX:(nbX+nbY)]
    l = uvlambda[(nbX+nbY):(nbX+nbY+nbK)]
    
    Phi_xy = phimat.dot(l.reshape(nbK,1)).reshape(nbX, nbY)
    mu_xy = np.exp(((Phi_xy - u_x).T-v_y).T/2)
    mu_x0 = np.exp(-u_x)
    mu_0y = np.exp(-v_y)
    
    val = sum(np.multiply(n_x,u_x))+sum(np.multiply(m_y,v_y))-np.sum(np.multiply(muhat_xy.values,Phi_xy), axis = (0,1)) + 2*np.sum(mu_xy, axis =(0,1)) + sum(mu_x0) + sum(mu_0y)
    
    return val

In [14]:
def grad_ObjFunc(uvlambda):
    u_x = uvlambda[0:nbX]
    v_y = uvlambda[nbX:(nbX+nbY)]
    l = uvlambda[(nbX+nbY):(nbX+nbY+nbK)]
    
    Phi_xy = phimat.dot(l.reshape(nbK,1)).reshape(nbX, nbY)
    mu_xy = np.exp(((Phi_xy - u_x).T-v_y).T/2)
    mu_x0 = np.exp(-u_x)
    mu_0y = np.exp(-v_y)
    
    grad_u = n_x - np.sum(mu_xy, axis = 0) - mu_x0
    grad_v = m_y - np.sum(mu_xy, axis = 1) - mu_0y
    grad_lambda = (mu_xy-muhat_xy.values).flatten()[:,None].T.dot(phimat)
    
    grad = np.concatenate((grad_u,grad_v,grad_lambda.flatten()))
    
    return grad

In [15]:
xs

array([[0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04,
        0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04,
        0.04, 0.04, 0.04],
       [0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08,
        0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08,
        0.08, 0.08, 0.08],
       [0.12, 0.12, 0.12, 0.12, 0.12, 0.12, 0.12, 0.12, 0.12, 0.12, 0.12,
        0.12, 0.12, 0.12, 0.12, 0.12, 0.12, 0.12, 0.12, 0.12, 0.12, 0.12,
        0.12, 0.12, 0.12],
       [0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16,
        0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16,
        0.16, 0.16, 0.16],
       [0.2 , 0.2 , 0.2 , 0.2 , 0.2 , 0.2 , 0.2 , 0.2 , 0.2 , 0.2 , 0.2 ,
        0.2 , 0.2 , 0.2 , 0.2 , 0.2 , 0.2 , 0.2 , 0.2 , 0.2 , 0.2 , 0.2 ,
        0.2 , 0.2 , 0.2 ],
       [0.24, 0.24, 0.24, 0.24, 0.24, 0.24, 0.24, 0.24, 0.24, 0.24, 0.24,
        0.24, 0.24, 0.24, 0.24, 0.24, 0.24, 0.24, 0

In [16]:
outcome = optimize.minimize(ObjFunc,method = 'CG',jac = grad_ObjFunc, x0 = np.repeat(0,nbX+nbY+nbK))

In [17]:
uvlambdahat =  outcome['x']
lambdahat = uvlambdahat[(nbX+nbY):(nbX+nbY+nbK)]
print(outcome)
print("")
print(ObjFunc(uvlambdahat))
print(lambdahat)

     fun: 7.677025691402801
     jac: array([ 4.88075801e-06,  5.10780308e-08, -5.40814185e-08, -9.67674359e-09,
       -3.27666407e-08, -5.03933471e-07, -1.01204451e-07, -6.63029291e-08,
       -6.00233391e-08, -5.64389548e-07,  9.76712940e-07,  1.22520487e-06,
        1.24756462e-06,  1.27584707e-06,  6.88245950e-07,  2.75181310e-07,
       -1.86412480e-07,  3.47244011e-07,  7.76230370e-07,  4.02940556e-08,
       -6.83570777e-07, -1.13239671e-06, -1.48543343e-06, -1.18186292e-06,
       -1.98637193e-06,  1.71723629e-06,  8.02963383e-07,  2.59413708e-07,
       -1.08526144e-07, -7.80600168e-07, -1.05835123e-06,  4.97217929e-07,
        1.18610346e-07,  1.95591308e-06,  2.88970690e-07,  5.54150734e-07,
        3.99675416e-07,  2.02151631e-06,  5.66475382e-07,  1.15734675e-06,
        2.94606296e-07,  1.68397986e-07,  1.32991993e-07,  1.41378739e-07,
       -3.87513667e-08, -2.92367924e-07, -1.25325454e-06, -1.78125071e-06,
       -1.47750538e-06, -4.15938391e-07, -1.59006568e-07,  5.7

In [18]:
print(lambdahat)

[-1.36756245 -6.4086041   4.60390741 -1.3049116 ]
